<a href="https://colab.research.google.com/github/HLCV-23/Inpainting-Detection/blob/Nicola/inpainting_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Related Work
## Inpainting
- https://arxiv.org/pdf/2102.12092.pdf  
  Dall-e
- https://arxiv.org/pdf/2112.10752.pdf  
  Stable-Diffusion

## GAN-generated images detection
- https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8397040  
  detection of images generated by GANs, no inpainting
- https://arxiv.org/pdf/2202.07145.pdf
  review of several GAN detection algorithms

## Diffusion-generated images detection
- https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=10095167  
  studies the performance of GAN-detection models on images generated by diffusion models

## Inpainting Detection
- https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9410590  
- https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9506778&tag=1  
  similar to our detection approach, but with random(?) masks. It compares the performance on deep learning and traditional inpainting techniques, which might be of interest for us (both for training and evaluation). We could also use the network architecture proposed here. Additional open-source dataset we might want to use.

## Image/Network Watermarking
- https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=650120  
  not directly related, but in watermarking, it is common to test the robustness of image watermarking techniques against common image preprocessing(rescaling, compression, etc). We might want to do that in our experiments too.
- https://www.usenix.org/system/files/conference/usenixsecurity18/sec18-adi.pdf
- https://tianweiz07.github.io/Papers/21-aamas.pdf  
- https://arxiv.org/abs/2305.20030 (Fourier Transform based technique to robustly watermark diffusion model outputs)

In [ ]:
!pip install transformers
!pip install diffusers
!pip install accelerate
!pip install xformers
import numpy as np
from diffusers import StableDiffusionInpaintPipeline
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from PIL import Image, ImageFilter

In [ ]:
device = torch.device("cuda:0")

mask_generator = pipeline("mask-generation", model="facebook/sam-vit-huge", device=device)

caption_generator = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

prompt_generator = pipeline("text-generation", model= model, tokenizer = tokenizer, device = device)

inpainting_model = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    torch_dtype=torch.float16,
    safety_checker=None,
    low_cpu_mem_usage = False
)

In [ ]:
# hugging face token is required to load dataset,
# go to https://huggingface.co/datasets/imagenet-1k/viewer/default/train
# and get a token in the account settings
!pip install datasets
!huggingface-cli login

In [ ]:
# download imagenet or scenes
from datasets import load_dataset, Dataset, Image
import pandas as pd
import os

dataset = load_dataset("imagenet-1k", use_auth_token = True, streaming = True, split = "train")  # set streaming to false for scenes!
dataset = iter(dataset.shuffle())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#"""
!mkdir drive/MyDrive/Imagenet_Inpainted
!touch drive/MyDrive/Imagenet_Inpainted/Prompts.csv
!mkdir drive/MyDrive/Imagenet_Inpainted/Images
!mkdir drive/MyDrive/Imagenet_Inpainted/Masks
!pwd
#"""

In [ ]:
# TODO: Save original image, save label
def generate_datapoint_single_mask(data, id, csv_data, path):
    data = data["image"].resize((512,512))
    image_area = 512*512
    masks = generator(data)["masks"]

    # Only masks with an area not too large or too small are eligible candidates for inpainting
    valid_masks = [mask for idx, mask in enumerate(masks) if (image_area * 0.025 <= mask.sum() <= image_area * 0.5)]

    if (valid_masks != []):
      choice = np.random.choice(np.arange(len(valid_masks)))
      mask = valid_masks[choice]

      mask_img = Image.fromarray((mask * 255).astype(np.uint8).squeeze())
      mask_img = mask_img.convert("RGB")

      # Generate prompt for inpainting model
      caption = image_to_text(data)[0]["generated_text"]
      prompts = gpt2_pipe(caption)[0]["generated_text"]
      prompt = prompts.split(",")[0]

      # inpaint
      inpainted_img = pipe(prompt=prompt, image=data, mask_image=mask_img, height=512, width=512).images[0]

      # get the edges of the mask
      edges = mask_img.filter(ImageFilter.FIND_EDGES).filter(ImageFilter.MaxFilter(7)).convert("1")

      # blur the inpainted image
      inpainted_img_blur = inpainted_img.filter(ImageFilter.GaussianBlur(radius = 1))

      # add the blured parts along the borders of the inpainted object to make it smoother
      inpainted_img_smooth = Image.composite(inpainted_img_blur, inpainted_img, edges).convert("RGB")

      inpainted_img_smooth.save(path + f"/Images/{id}.png")
      mask_img.save(path + f"/Masks/{id}.png")
      csv_data.at[id, "Prompt"] = prompt
      print(f"Saved processed image {id}.jpeg")

    else:
      print(f"No valid mask could be generated. Skipping image {id}.")

In [ ]:
path = "drive/MyDrive/Imagenet_Inpainted"

try:
  csv_data = pd.read_csv(path + "/Prompts.csv")
except:
  with open(path + "/Prompts.csv", "w+") as f:
    f.write("idx, Prompt")
  csv_data = pd.read_csv(path + "/Prompts.csv")

for id in range(0,100):
    generate_datapoint_single_mask(next(dataset), id, csv_data, path)

csv_data.to_csv(path + "/Prompts.csv")